In [ ]:
# verify K-Center-greedy algorithm in coreset query strategy

In [1]:
# import modules
import sys
import numpy as np
import os
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree
from scipy.spatial import distance_matrix

from deepvisualinsight.MMS import MMS
from deepvisualinsight import utils

In [2]:
# helper function
def greedy_k_center(labeled, unlabeled, amount):

    greedy_indices = []

    # get the minimum distances between the labeled and unlabeled examples (iteratively, to avoid memory issues):
    min_dist = np.min(distance_matrix(labeled[0, :].reshape((1, labeled.shape[1])), unlabeled), axis=0)
    min_dist = min_dist.reshape((1, min_dist.shape[0]))
    for j in range(1, labeled.shape[0], 1000):
        if j + 1000 < labeled.shape[0]:
            dist = distance_matrix(labeled[j:j+1000, :], unlabeled)
        else:
            dist = distance_matrix(labeled[j:, :], unlabeled)
        min_dist = np.vstack((min_dist, np.min(dist, axis=0).reshape((1, min_dist.shape[1]))))
        min_dist = np.min(min_dist, axis=0)
        min_dist = min_dist.reshape((1, min_dist.shape[0]))

    # iteratively insert the farthest index and recalculate the minimum distances:
    farthest = np.argmax(min_dist)
    greedy_indices.append(farthest)
    for i in range(amount-1):
        dist = distance_matrix(unlabeled[greedy_indices[-1], :].reshape((1,unlabeled.shape[1])), unlabeled)
        min_dist = np.vstack((min_dist, dist.reshape((1, min_dist.shape[1]))))
        min_dist = np.min(min_dist, axis=0)
        min_dist = min_dist.reshape((1, min_dist.shape[0]))
        farthest = np.argmax(min_dist)
        greedy_indices.append(farthest)

    return np.array(greedy_indices)

In [3]:
content_path = "E:\\DVI_exp_data\\active_learning\\coreset"
sys.path.append(content_path)

from Model.model import *
net = ResNet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

mms = MMS(content_path, net, 1, 20, 1, 512, 10, classes, cmap="tab10", resolution=100, neurons=256,
          verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

Finish loading content!


In [4]:
# high-epsilon
epsilons = np.zeros(9)
for n_epoch in range(1,10, 1):
    new_index_file = os.path.join(mms.model_path, "Epoch_{:d}".format(n_epoch+1), "index.json")
    new_index = utils.load_labelled_data_index(new_index_file)

    training_data = mms.training_data
    train_data = mms.get_representation_data(n_epoch, training_data)

    index_file = os.path.join(mms.model_path, "Epoch_{:d}".format(n_epoch), "index.json")
    index = utils.load_labelled_data_index(index_file)

    curr = train_data[new_index]

    high_tree = KDTree(curr)
    dists, _ = high_tree.query(train_data, k=1)

    dists = dists.squeeze()
    epsilons[n_epoch-1] = dists.max()
epsilons

array([4.02150939, 3.9213272 , 2.92771248, 3.52480613, 3.02320138,
       3.52242175, 2.75349766, 3.29114047, 3.02116381])

In [6]:
# high-epsilon
epsilons = np.zeros(9)
for n_epoch in range(1, 10, 1):
    new_index_file = os.path.join(mms.model_path, "Epoch_{:d}".format(n_epoch+1), "index.json")
    new_index = utils.load_labelled_data_index(new_index_file)
    training_data = mms.training_data
    train_data = mms.get_representation_data(n_epoch, training_data)

    index_file = os.path.join(mms.model_path, "Epoch_{:d}".format(n_epoch), "index.json")
    index = utils.load_labelled_data_index(index_file)

    repr_2d = mms.batch_project(train_data, n_epoch)
    curr = repr_2d[new_index]
    high_tree = KDTree(curr)
    dists, _ = high_tree.query(repr_2d, k=1)
    dists = dists.squeeze()
    epsilons[n_epoch-1] = dists.max()
epsilons

Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3B3ED860>
Instructions for updating:
Use tf.identity instead.
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3E5F6278>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3E566320>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C0384E6D8>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C08209AC8>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C75F694E0>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C7F9638D0>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C982B7F60>
Keras encoder model 

array([0.70202433, 0.44740197, 1.51597259, 1.36008101, 1.50354327,
       1.67582399, 2.22395194, 2.75586312, 1.94168804])

Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3AC4FE80>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022CB12B29B0>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3E4A1898>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3B8D1D30>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3E58FF98>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C081EA160>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3CF8CCF8>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C75F7A4A8>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequenti

array([0.70202433, 0.44740197, 1.51597259, 1.36008101, 1.50354327,
       1.67582399, 2.22395194, 2.75586312, 1.94168804])

In [9]:
epsilons = np.zeros(9)
for n_epoch in range(1, 10, 1):
    training_data = mms.training_data
    train_data = mms.get_representation_data(n_epoch, training_data)

    index_file = os.path.join(mms.model_path, "Epoch_{:d}".format(n_epoch), "index.json")
    index = utils.load_labelled_data_index(index_file)

    repr_2d = mms.batch_project(train_data, n_epoch)
    unl_idx = np.setdiff1d(np.arange(50000),np.array(index))
    new_index = greedy_k_center(repr_2d[index], repr_2d[unl_idx], 1000)
    all_index = index + new_index.tolist()

    high_tree = KDTree(repr_2d[all_index])
    dists, _ = high_tree.query(repr_2d, k=1)
    dists = dists.squeeze()
    epsilons[n_epoch-1] = dists.max()
epsilons

Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3E7A6CC0>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3CDE5208>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C7B6600F0>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3D0D1BE0>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022CE757F0B8>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022C3D094E48>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022CE741B048>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022CE745C208>
Keras encoder model loaded from <tensorflow.python.keras.engine.sequenti

array([0.76349818, 0.75984228, 1.51672642, 1.97005472, 1.49694344,
       2.50808536, 2.50276392, 2.75586312, 2.08008018])